In [ ]:
import cv2
import tensorflow as tf

input_image = tf.constant(cv2.imread('../input/landmark-retrieval-2020/train/0/0/0/0000059611c7d079.jpg'), dtype=tf.uint8)

In [ ]:
model = tf.saved_model.load("../input/google-2020-baseline/∩╗┐basline")

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model1 = model
        self.f1 = self.model1.signatures["serving_default"]
        
    @tf.function(input_signature=[
      tf.TensorSpec(shape=[None, None, 3], dtype=tf.uint8, name='input_image')
    ])
    def call(self, input_image):
        output_tensors = {}
        
        width = tf.cast(tf.shape(input_image)[0], dtype=tf.float32)
        height = tf.cast(tf.shape(input_image)[1], dtype=tf.float32)
        i1 = tf.cast(tf.image.resize(input_image, [tf.cast(pow(2,0.5) * width, dtype=tf.int32), tf.cast(pow(2,0.5) * height, dtype = tf.int32)]), dtype = input_image.dtype)
        
        output_tensors['global_descriptor'] = tf.identity(self.f1(i1)['global_descriptor'], name='global_descriptor')
        return output_tensors

In [ ]:
m = MyModel() #creating our model instance

served_function = m.call
tf.saved_model.save(
      m, export_dir="./my_model", signatures={'serving_default': served_function})

In [ ]:
!saved_model_cli show --dir "../input/google-2020-baseline/∩╗┐basline" --all

In [ ]:
!saved_model_cli show --dir ./my_model/ --all

In [ ]:
imported3 = tf.saved_model.load("./my_model")
f3 = imported3.signatures["serving_default"]

f3(input_image)

In [ ]:
!ls ./my_model/variables

In [ ]:
from zipfile import ZipFile

with ZipFile('submission.zip','w') as zip:           
    zip.write('./my_model/saved_model.pb', arcname='saved_model.pb')
    zip.write('./my_model/variables/variables.data-00000-of-00001', arcname='variables/variables.data-00000-of-00001') 
    zip.write('./my_model/variables/variables.index', arcname='variables/variables.index') 